In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow INFO and WARNING logs
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import cv2

In [2]:
# Path to dataset
DATASET_PATH = r"C:\Users\ASUS\Downloads\dataset-resized"  # Update with your dataset path
CATEGORIES = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]  # Update based on your folder names


In [3]:
# Load and preprocess data
def load_data():
    images, labels = [], []
    for label, category in enumerate(CATEGORIES):
        folder_path = os.path.join(DATASET_PATH, category)
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            try:
                img = tf.keras.utils.load_img(img_path, target_size=(128, 128))  # Resize images
                img_array = tf.keras.utils.img_to_array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)

# Load dataset
images, labels = load_data()
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Train Data: {X_train.shape}, Test Data: {X_test.shape}")

Train Data: (2021, 128, 128, 3), Test Data: (506, 128, 128, 3)


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Build the model
def build_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False  # Freeze base model
    
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(len(CATEGORIES), activation='softmax')
    ])
    return model

model = build_model()
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")



Epoch 1/10


64/64 [==============================] - 37s 474ms/step - loss: 1.4427 - accuracy: 0.6037 - val_loss: 0.7426 - val_accuracy: 0.7075
Epoch 2/10
64/64 [==============================] - 26s 405ms/step - loss: 0.6070 - accuracy: 0.7778 - val_loss: 0.7088 - val_accuracy: 0.7806
Epoch 3/10
64/64 [==============================] - 26s 412ms/step - loss: 0.4078 - accuracy: 0.8436 - val_loss: 0.5880 - val_accuracy: 0.8142
Epoch 4/10
64/64 [==============================] - 26s 408ms/step - loss: 0.2780 - accuracy: 0.8986 - val_loss: 0.6693 - val_accuracy: 0.7945
Epoch 5/10
64/64 [==============================] - 26s 408ms/step - loss: 0.2303 - accuracy: 0.9223 - val_loss: 0.6729 - val_accuracy: 0.7945
Epoch 6/10
64/64 [==============================] - 33s 520ms/step - loss: 0.1951 - accuracy: 0.9292 - val_loss: 0.6407 - val_accuracy: 0.8182
Epoch 7/10
64/64 [==============================] - 26s 405ms/step - loss: 0.1784 - accuracy: 0.9292 - val_loss: 0.6762 - val_accuracy: 0.

In [5]:
# Save the model in the SavedModel format
model.save("waste_classifier")

# Load the model for deployment
deployed_model = tf.keras.models.load_model("waste_classifier")

# Real-time classification function
def classify_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(128, 128))
    img_array = tf.keras.utils.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    predictions = deployed_model.predict(img_array)
    predicted_class = CATEGORIES[np.argmax(predictions)]
    confidence = np.max(predictions)
    return predicted_class, confidence

INFO:tensorflow:Assets written to: waste_classifier\assets


INFO:tensorflow:Assets written to: waste_classifier\assets


In [6]:
# Real-time waste classification with OpenCV
def live_classification():
    cap = cv2.VideoCapture(0)  # Use webcam
    if not cap.isOpened():
        print("Error: Webcam not accessible!")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        # Preprocess frame
        resized_frame = cv2.resize(frame, (128, 128))
        normalized_frame = resized_frame / 255.0
        frame_array = np.expand_dims(normalized_frame, axis=0)

        # Predict
        predictions = deployed_model.predict(frame_array)
        predicted_class = CATEGORIES[np.argmax(predictions)]
        confidence = np.max(predictions)

        # Display results
        cv2.putText(frame, f"{predicted_class} ({confidence:.2f})", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Waste Classification", frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run live classification
live_classification()

# Convert the TensorFlow model to TensorFlow Lite
try:
    converter = tf.lite.TFLiteConverter.from_saved_model("waste_classifier")
    converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Optional: Enable optimizations
    tflite_model = converter.convert()
    with open("waste_classifier.tflite", "wb") as f:
        f.write(tflite_model)
    print("Model successfully converted to TensorFlow Lite and saved as 'waste_classifier.tflite'.")
except Exception as e:
    print(f"Error during model conversion: {e}")

1/1 [==============================] - 0s 44ms/step


1/1 [==============================] - 0s 42ms/step
Model successfully converted to TensorFlow Lite and saved as 'waste_classifier.tflite'.
